# So alfa beta

In [19]:
from plotly import graph_objects as go
import ipywidgets as widgets
from collections import deque
from copy import deepcopy
import time

In [20]:
node_count = 0

In [21]:
def minimaxAlphaBeta(depth, board, alpha, beta, player, node_count=0):
    if depth == 0:
        return board.score(), node_count

    moves = board.available_moves()
    if player:
        bestMove = -9999
        for move in moves:
            board.push(move)
            value, node_count = minimaxAlphaBeta(depth - 1, board, alpha, beta, not player, node_count + 1)
            bestMove = max(bestMove, value)
            board.pop()
            alpha = max(alpha, bestMove)
            if beta <= alpha:
                break
        return bestMove, node_count
    else:
        bestMove = 9999
        for move in moves:
            board.push(move)
            value, node_count = minimaxAlphaBeta(depth - 1, board, alpha, beta, not player, node_count + 1)
            bestMove = min(bestMove, value)
            board.pop()
            beta = min(beta, bestMove)
            if beta <= alpha:
                break
        return bestMove, node_count

def minimaxAlphaBetaRoot(depth, board, player):
    moves = board.available_moves()
    scores = {}
    bestMove = -9999 if player else 9999
    alpha = -10000
    beta = 10000
    node_count = 0

    for move in moves:
        board.push(move)
        value, node_count = minimaxAlphaBeta(depth - 1, board, alpha, beta, not player, node_count + 1)
        scores[move] = value
        board.pop()
        if player and value >= bestMove:
            bestMove = value
            bestMoveFound = move
        elif not player and value <= bestMove:
            bestMove = value
            bestMoveFound = move

    return bestMoveFound, moves, scores, node_count




In [22]:

class Board:
    def __init__(self, table, turn=True):
        self.table = deepcopy(table)
        self.queue = []
        self.turn = turn 
        
    def push(self, move):
        self.queue.append(move)
        self.update_table()
        self.turn = not self.turn
        
    def pop(self):
        move = self.queue.pop()
        self.update_table(move)
        self.turn = not self.turn
        return move
        
    def peek(self):
        return self.queue[-1]
    
    def update_table(self, move=None):
        if move is not None:
            old, new = move
            self.table[old[0]][old[1]] = not self.turn
            self.table[new[0]][new[1]] = None if old[1] == new[1] else self.turn
        else:
            old, new = self.peek()
            self.table[old[0]][old[1]] = None
            self.table[new[0]][new[1]] = self.turn
            
    def legal_moves(self, x, y):
        x += -1 if self.turn else 1
        moves = []
        for i in range(-1, 2):
            if 0 <= x < M and 0 <= y + i < N:
                if self.table[x][y + i] is None and i == 0:
                    moves.append((x, y))
                elif self.table[x][y + i] == (not self.turn) and i != 0:
                    moves.append((x, y + i))
        return moves
    
    def available_moves(self):
        moves = []
        for x in range(M):
            for y in range(N):
                if self.table[x][y] == self.turn:
                    for move in self.legal_moves(x, y):
                        moves.append(((x, y), move))
        return moves
    
    def score(self):
        score = 0
        for x in range(M):
            for y in range(N):
                if self.table[x][y] is not None:
                    score += M - x if self.table[x][y] else -(x + 1)
                    if x == 0 and self.table[x][y]:
                        return 9999
                    elif x == M - 1 and not self.table[x][y]:
                        return -9999
        score += len([i for row in self.table for i in row if i]) - len([i for row in self.table for i in row if i is False])
        return score

In [23]:
def posible_moves(x, y):
    moves = []
    if TABLE[x][y] is None:
        return moves
    for i in range(-1, 2):
        if 0 <= x - 1 < M and 0 <= y + i < N:
            if TABLE[x - 1][y + i] is None and i == 0:
                moves.append((x - 1, y))
            elif TABLE[x - 1][y + i] is False and i != 0:
                moves.append((x - 1, y + i))
    return moves

def game_end(board):
    if False in board.table[M - 1]:
        return "Red"
    elif True in board.table[0]:
        return "Blue"
    if not board.available_moves():
        return "Red" if board.turn else "Blue"
    return None

In [24]:
debug = True

def update_fig(trace, points, selector):
    global TABLE, node_count
    
    board = Board(TABLE)
    move, moves, scores, node_count = minimaxAlphaBetaRoot(4, board, True)
    if debug: 
        print(f"AI (Blue) move: {move}, Scores: {scores}, Node count: {node_count}")

    board.push(move)
    TABLE = board.table
    colors = ['#121212' if n is None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.data[0].marker['color'] = colors

    if game_end(Board(TABLE)) is not None:
        text.value = f'<font size=4> {game_end(Board(TABLE))} wins !! Nodes evaluated: {node_count}'
        fig.data[0].on_click(None)
        return 
    
    time.sleep(1) # AI Part
    board = Board(TABLE, False)
    move, moves, scores, node_count =minimaxAlphaBetaRoot(5, board, False)
    if debug: 
        print(f"AI (Red) move: {move}, Scores: {scores}, Node count: {node_count}")

    board.push(move)
    TABLE = board.table
    colors = ['#121212' if n is None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.data[0].marker['color'] = colors

    if game_end(Board(TABLE)) is not None:
        text.value = f'<font size=4> {game_end(Board(TABLE))} wins !! Nodes evaluated: {node_count}'
        fig.data[0].on_click(None)
        return
    
def create_fig():
    fig = go.FigureWidget()
    x = [x for y in range(M) for x in range(N)]
    y = [y for y in range(M) for x in range(N)]
    colors = ['#121212' if n is None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.add_scatter(x=x, y=y, mode='markers', marker_size=(M + 1) * 10 + N + 2, 
        marker_symbol='square', marker_color=colors, hoverinfo='none')
    fig.data[0].on_click(update_fig)
    fig.update_xaxes(range=[-0.5, N - 0.5], dtick=1, side='top', visible=False, autorange=False)
    fig.update_yaxes(range=[-0.5, M - 0.5], dtick=1, autorange='reversed', visible=False)
    fig.update_layout(width=80 * N, height=80 * N, showlegend=False, margin={'r':0, 'l':0, 't':0, 'b':0}, plot_bgcolor='#212121')

    return fig

N = 6
M = 5
TABLE = [
    [False,] * N,
    [None,] * N,
    [None,] * N,
    [None,] * N,
    [True,] * N,
]
fig = create_fig()
text = widgets.HTML(value='<font size=4> Hexapawn')
display(widgets.VBox([fig, text]))
node_count = 0


    'data': [{'hoverinfo': 'none',
              'marker': {'color': [#d62728, #…

# Bez alfa beta

In [25]:
node_count = 0

In [26]:
def minimax(depth, board, player, node_count=0):
    if depth == 0:
        return board.score(), node_count

    moves = board.available_moves()
    if player:
        bestMove = -9999
        for move in moves:
            board.push(move)
            value, node_count = minimax(depth - 1, board, not player, node_count + 1)
            bestMove = max(bestMove, value)
            board.pop()
        return bestMove, node_count
    else:
        bestMove = 9999
        for move in moves:
            board.push(move)
            value, node_count = minimax(depth - 1, board, not player, node_count + 1)
            bestMove = min(bestMove, value)
            board.pop()
        return bestMove, node_count

def minimaxRoot(depth, board, player):
    moves = board.available_moves()
    scores = {}
    bestMove = -9999 if player else 9999
    node_count = 0

    for move in moves:
        board.push(move)
        value, node_count = minimax(depth - 1, board, not player, node_count + 1)
        scores[move] = value
        board.pop()
        if player and value >= bestMove:
            bestMove = value
            bestMoveFound = move
        elif not player and value <= bestMove:
            bestMove = value
            bestMoveFound = move

    return bestMoveFound, moves, scores, node_count


In [27]:

class Board:
    def __init__(self, table, turn=True):
        self.table = deepcopy(table)
        self.queue = []
        self.turn = turn 
        
    def push(self, move):
        self.queue.append(move)
        self.update_table()
        self.turn = not self.turn
        
    def pop(self):
        move = self.queue.pop()
        self.update_table(move)
        self.turn = not self.turn
        return move
        
    def peek(self):
        return self.queue[-1]
    
    def update_table(self, move=None):
        if move is not None:
            old, new = move
            self.table[old[0]][old[1]] = not self.turn
            self.table[new[0]][new[1]] = None if old[1] == new[1] else self.turn
        else:
            old, new = self.peek()
            self.table[old[0]][old[1]] = None
            self.table[new[0]][new[1]] = self.turn
            
    def legal_moves(self, x, y):
        x += -1 if self.turn else 1
        moves = []
        for i in range(-1, 2):
            if 0 <= x < M and 0 <= y + i < N:
                if self.table[x][y + i] is None and i == 0:
                    moves.append((x, y))
                elif self.table[x][y + i] == (not self.turn) and i != 0:
                    moves.append((x, y + i))
        return moves
    
    def available_moves(self):
        moves = []
        for x in range(M):
            for y in range(N):
                if self.table[x][y] == self.turn:
                    for move in self.legal_moves(x, y):
                        moves.append(((x, y), move))
        return moves
    
    def score(self):
        score = 0
        for x in range(M):
            for y in range(N):
                if self.table[x][y] is not None:
                    score += M - x if self.table[x][y] else -(x + 1)
                    if x == 0 and self.table[x][y]:
                        return 9999
                    elif x == M - 1 and not self.table[x][y]:
                        return -9999
        score += len([i for row in self.table for i in row if i]) - len([i for row in self.table for i in row if i is False])
        return score

In [28]:
def posible_moves(x, y):
    moves = []
    if TABLE[x][y] is None:
        return moves
    for i in range(-1, 2):
        if 0 <= x - 1 < M and 0 <= y + i < N:
            if TABLE[x - 1][y + i] is None and i == 0:
                moves.append((x - 1, y))
            elif TABLE[x - 1][y + i] is False and i != 0:
                moves.append((x - 1, y + i))
    return moves

def game_end(board):
    if False in board.table[M - 1]:
        return "Red"
    elif True in board.table[0]:
        return "Blue"
    if not board.available_moves():
        return "Red" if board.turn else "Blue"
    return None

In [29]:
debug = True

def update_fig(trace, points, selector):
    global TABLE, node_count
    
    board = Board(TABLE)
    move, moves, scores, node_count = minimaxRoot(4, board, True)
    if debug: 
        print(f"AI (Blue) move: {move}, Scores: {scores}, Node count: {node_count}")

    board.push(move)
    TABLE = board.table
    colors = ['#121212' if n is None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.data[0].marker['color'] = colors

    if game_end(Board(TABLE)) is not None:
        text.value = f'<font size=4> {game_end(Board(TABLE))} wins !! Nodes evaluated: {node_count}'
        fig.data[0].on_click(None)
        return 
    
    time.sleep(1) # AI Part
    board = Board(TABLE, False)
    move, moves, scores, node_count =minimaxRoot(5, board, False)
    if debug: 
        print(f"AI (Red) move: {move}, Scores: {scores}, Node count: {node_count}")

    board.push(move)
    TABLE = board.table
    colors = ['#121212' if n is None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.data[0].marker['color'] = colors

    if game_end(Board(TABLE)) is not None:
        text.value = f'<font size=4> {game_end(Board(TABLE))} wins !! Nodes evaluated: {node_count}'
        fig.data[0].on_click(None)
        return
    
def create_fig():
    fig = go.FigureWidget()
    x = [x for y in range(M) for x in range(N)]
    y = [y for y in range(M) for x in range(N)]
    colors = ['#121212' if n is None else '#1f77b4' if n else '#d62728' for row in TABLE for n in row]
    fig.add_scatter(x=x, y=y, mode='markers', marker_size=(M + 1) * 10 + N + 2, 
        marker_symbol='square', marker_color=colors, hoverinfo='none')
    fig.data[0].on_click(update_fig)
    fig.update_xaxes(range=[-0.5, N - 0.5], dtick=1, side='top', visible=False, autorange=False)
    fig.update_yaxes(range=[-0.5, M - 0.5], dtick=1, autorange='reversed', visible=False)
    fig.update_layout(width=80 * N, height=80 * N, showlegend=False, margin={'r':0, 'l':0, 't':0, 'b':0}, plot_bgcolor='#212121')

    return fig

N = 6
M = 5
TABLE = [
    [False,] * N,
    [None,] * N,
    [None,] * N,
    [None,] * N,
    [True,] * N,
]
fig = create_fig()
text = widgets.HTML(value='<font size=4> Hexapawn')
display(widgets.VBox([fig, text]))
node_count = 0


    'data': [{'hoverinfo': 'none',
              'marker': {'color': [#d62728, #…